In [1]:
from pathlib import Path
import pprint
import json
import numpy as np
from collections import defaultdict
import pandas as pd
from IPython.display import display, HTML
from ipywidgets import interact
from scipy.stats import t
from scipy.stats import ttest_ind, ttest_1samp

In [4]:
folders = list(sorted(Path(f"../../jass_mu_zero/scripts/agents_eval_results/").glob("*"), reverse=False))
folders 

[PosixPath('../../jass_mu_zero/scripts/agents_eval_results/results')]

# Experiment wise tables

In [9]:
@interact(folder=folders)
def f(folder):
    print(40*"-")
    print(folder, "(p1: mu<= 50%, p2: mu = 50%)")
    print(40*"-")
    relevant_data = defaultdict(lambda: {})
    files = folder.glob("tmp/*.json")

    for file in files:
        with open(str(file), "r") as f:
            data = json.load(f)
        
        name =  Path(file).name.replace(".json", "")
        opponent = name.split("-vs-")[0].replace("-", " ")
        agent = name.split("-vs-")[1].replace("-", " ")

        key = [x for x in data.keys() if not x.__contains__("DMCTS") and not x.__contains__("Policy-S")
               and not x.__contains__("MCTS-100-S") and not x.__contains__("PMCTS") and not x.__contains__("mean")
               and not x.__contains__("Policy-B") and not x.__contains__("DQN") and not x.__contains__("MCTS-100-3")][0]

        relevant_data[agent][opponent] = data[key]
            
    data = {k: {x: np.array(y) for x, y in v.items()} for k,v in relevant_data.items()}
    data = {k: {x: y[:4*(y.shape[0]//4)].reshape(-1, 4) for x, y in v.items()} for k,v in data.items()}
    data = {k: {x: y 
                for x, y in sorted(v.items(), key=lambda item: item[0])}
            for k, v in sorted(data.items(), key=lambda item: item[0], reverse=True)} 
    
    print_data = {}
    
    for agent in data:
        print_data[agent] = {}
        for x, y in data[agent].items():
            mu =y.mean(axis=1).mean()
            ste = y.mean(axis=1).std() / np.sqrt(y.shape[0])
            p1 = ttest_1samp(y.reshape(-1), 0.5, nan_policy='omit', alternative='greater').pvalue
            p2 = ttest_1samp(y.reshape(-1), 0.5, nan_policy='omit', alternative='two-sided').pvalue
            
            print_data[agent][x] = f"{mu.round(5)}+-{ste.round(5)} {y.shape}, p1={p1.round(5)}, p2={p2.round(5)}"
    
    df = pd.DataFrame(print_data).T
    
    display(HTML(df.to_html()))
    
    print(40*"-")
    print("Compare distributions of row agents:")
    print(40*"-")
    
    @interact(agent1=df.index, agent2=df.index)
    def f(agent1, agent2):
        
        for opponent in df.columns:
            if opponent in data[agent1] and opponent in data[agent2]:
                length = min(data[agent1][opponent].reshape(-1).shape[0], data[agent2][opponent].reshape(-1).shape[0])
                print(opponent, length)
                print(ttest_ind(data[agent1][opponent].reshape(-1)[:length],
                                data[agent2][opponent].reshape(-1)[:length], nan_policy='omit'))

        

interactive(children=(Dropdown(description='folder', options=(PosixPath('../../jass_mu_zero/scripts/agents_eva…